# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [13]:
import sys
import random
import functools
import pprint

from leap_ec.individual import Individual

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import perform_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[0, 1, 1, 1, 1], [0, 0, 0, 1, 0], [0, 0, 1, 0, 1], [0, 1, 1, 1, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 0, 1, 0, 1], [0, 0, 1, 1, 1], [1, 0, 0, 0, 1], [0, 0, 1, 0, 0]]
1 [[1, 1, 0, 0, 0], [0, 0, 1, 0, 1], [0, 1, 1, 0, 1]]
2 [[1, 1, 1, 1, 0]]
3 [[0, 1, 1, 1, 1]]
4 [[1, 1, 1, 0, 0], [1, 1, 0, 0, 1]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [5]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[2, 5], [0, 7], [2, 1], [0, 4]]
1 [[3, 0], [0, 5], [1, 5]]
2 [[3, 6]]
3 [[1, 7]]
4 [[3, 4], [3, 1]]


In [15]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=perform_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[1, 0], [1, 1]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [17]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pprint.pformat(s, indent=2))

0 [ [-1.3662518010924898, -0.43688011839615704, 3.740879916999793],
  [4.005918193012968, -0.5093705011751135, 1.882354114875536]]
1 [ [-5.033511627167246, 0.4353470869921525, -8.611771556359908],
  [-0.7109068760979609, -0.044179378966994864, -7.7612617860075535]]
2 [ [-4.010514168217261, -0.6746659326697635, 4.332345274842096],
  [4.3292182242695825, 0.006425949368952466, 2.126691837345163]]
3 [ [-4.170043418174771, 0.8897661108883439, -8.1857500489624],
  [-4.342640876053443, -0.6467017823491039, -2.2319173798840968],
  [1.0500790266180449, -0.3442422813111843, -8.803839020636818]]
4 [ [0.9191569154081849, -0.19904618766776272, -3.7447083972247803],
  [3.032966802992026, -0.7318116753000388, -9.334580706619706]]
5 [ [-0.3188159178124286, 0.6425298776990858, -1.9304348076175195],
  [-3.6845779466718795, -0.6176313979199692, 8.829406166062899],
  [1.037385624550728, 0.6542008853264791, -4.133312099402913],
  [0.2722551291008948, 0.4777402894014464, 4.978340736319334],
  [4.88378519169

Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [ ]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=perform_mutate_bitflip))
print('mutated:', mutated)